In [1]:

import os, json, gc
import numpy as np, pandas as pd, math, yaml

from pprint import pprint
from datetime import datetime as dt

gc.collect()


0

In [2]:

loc_exp = 'C:\\Users\\karan\\Documents\\Cricket-Analytics\\IPL\\Data\\exp\\'

#location_json = 'C:\\Users\\karan\\Documents\\Cricket-Analytics\\IPL\\Data\\ipl_json_2023_05_15\\'
location_yaml = "C:\\Users\\karan\\Documents\\Cricket-Analytics\\IPL\\Data\\ipl_yaml_2023_05_18\\"


In [3]:

print(loc_exp)
print()
#print(location_json)
#print()
print(location_yaml)
print()
print(dt.now().date())


C:\Users\karan\Documents\Cricket-Analytics\IPL\Data\exp\

C:\Users\karan\Documents\Cricket-Analytics\IPL\Data\ipl_yaml_2023_05_18\

2023-05-20


#### IMPORT FILE IF YAML

In [5]:

print(dt.now())

df_list = []

# Define a dictionary to map team names to abbreviations
team_abbreviations = {
    'Kolkata Knight Riders': 'KKR',
    'Royal Challengers Bangalore': 'RCB',
    'Punjab Kings' : 'PBKS',
    'Chennai Super Kings' : 'CSK',
    'Mumbai Indians' : 'MI',
    'Gujarat Titans' : 'GT',
    'Sunrisers Hyderabad': 'SRH',
    'Lucknow Super Giants': 'LSG',
    'Rajasthan Royals' : 'RR',
    'Delhi Capitals' : 'DC' 
}

# Initialize DataFrame to hold all ball-by-ball information
all_data = pd.DataFrame()

# Specify the directory where your YAML files are stored
directory = location_yaml

# Initialize a dictionary to track previous batting orders for each team
previous_bat_orders = {}

# Loop over all YAML files in the directory
for filename in os.listdir(directory):
    if filename.endswith('.yaml') and int(filename[:-5]) >= 1359475:  # Check if the filename (without extension) is larger than 1359475
        print()
        print(filename)
        with open(os.path.join(directory, filename)) as file:
            data = yaml.safe_load(file)

            # Initialize list to hold ball-by-ball information
            balls_data = []

            # Initialize dictionaries to track batting order for each team
            batting_order = {team_abbreviations[team]: {} for team in data['info']['teams']}
            order_counter = {team_abbreviations[team]: 1 for team in data['info']['teams']}

            # Traverse through the innings
            for inning in data['innings']:
                for inning_name, inning_info in inning.items():
                    print(inning_info['team'])
                    team_batting = team_abbreviations[inning_info['team']]
                    team_bowling = team_abbreviations[data['info']['teams'][1] if inning_info['team'] == data['info']['teams'][0] else data['info']['teams'][0]]
                    print(f'{team_batting} vs {team_bowling}')
                    for delivery in inning_info['deliveries']:
                        for delivery_num, delivery_info in delivery.items():
                            batsman = delivery_info['batsman']
                            if batsman not in batting_order[team_batting]:
                                batting_order[team_batting][batsman] = order_counter[team_batting]
                                order_counter[team_batting] += 1
                            delivery_info['bat_order'] = batting_order[team_batting][batsman]
                            delivery_info['b.num'] = delivery_num
                            delivery_info['inning'] = inning_name
                            delivery_info['team_batting'] = team_batting
                            delivery_info['team_bowling'] = team_bowling
                            delivery_info['city'] = data['info']['city']
                            delivery_info['date'] = data['info']['dates'][0]
                            delivery_info['competition'] = data['info']['competition']
                            delivery_info['match_type'] = data['info']['match_type']
                            delivery_info['batsman_runs'] = delivery_info['runs']['batsman']
                            delivery_info['extras'] = delivery_info['runs']['extras']
                            delivery_info['wkt'] = 1 if 'wicket' in delivery_info else 0
                            delivery_info.pop('runs', None)  # Remove the 'runs' column
                            delivery_info.pop('wicket', None)  # Remove the 'wicket' column
                            delivery_info.pop('non_striker', None)  # Remove the 'non_striker' column
                            balls_data.append(delivery_info)
                        
            # Include players who didn't bat in the game
            for team in data['info']['teams']:
                team_abbr = team_abbreviations[team]
                for player in data['info']['players'][team]:
                    if player not in batting_order[team_abbr]:
                        if team_abbr in previous_bat_orders and player in previous_bat_orders[team_abbr]:
                            batting_order[team_abbr][player] = max(previous_bat_orders[team_abbr][player])
                        else:
                            batting_order[team_abbr][player] = order_counter[team_abbr]
                            order_counter[team_abbr] += 1
                        dummy_info = {
                            'bat_order': batting_order[team_abbr][player],
                            'b.num': 0,
                            'inning': 'N/A',
                            'team_batting': team_abbr,
                            'team_bowling': 'N/A',
                            'city': data['info']['city'],
                            'date': data['info']['dates'][0],
                            'competition': data['info']['competition'],
                            'match_type': data['info']['match_type'],
                            'batsman': player,
                            'batsman_runs': 0,
                            'extras': 0,
                            'wkt': 0
                        }
                        balls_data.append(dummy_info)

            # Create DataFrame from the list of dictionaries
            df = pd.DataFrame(balls_data)
            print(df.columns.values.tolist())

            # Rename the specified columns
            df.rename(columns={
                'batsman_runs': 'tr',
                'team_batting': 'tm_bat',
                'team_bowling': 'tm_bow',
                'batsman': 'bat',
                'bowler': 'bow',
                'match_type': 'mtype'
            }, inplace=True)
            
            #display(df)
            #last_index = df.index

            print(df.shape)
            print(all_data.shape)
            print(df.columns.values.tolist())
            print(all_data.columns.values.tolist())
            
            # Append this DataFrame to the master DataFrame
            all_data = pd.concat([all_data, df], ignore_index=True)            
            del df

print()
print(dt.now())



2023-05-20 13:08:39.392042

1359475.yaml
Chennai Super Kings
CSK vs GT
Gujarat Titans
GT vs CSK
['bowler', 'batsman', 'bat_order', 'b.num', 'inning', 'team_batting', 'team_bowling', 'city', 'date', 'competition', 'match_type', 'batsman_runs', 'extras', 'wkt', 'replacements']
(251, 15)
(0, 0)
['bow', 'bat', 'bat_order', 'b.num', 'inning', 'tm_bat', 'tm_bow', 'city', 'date', 'competition', 'mtype', 'tr', 'extras', 'wkt', 'replacements']
[]

1359476.yaml
Punjab Kings
PBKS vs KKR
Kolkata Knight Riders
KKR vs PBKS
['bowler', 'batsman', 'bat_order', 'b.num', 'inning', 'team_batting', 'team_bowling', 'city', 'date', 'competition', 'match_type', 'batsman_runs', 'extras', 'wkt', 'replacements']
(230, 15)
(251, 15)
['bow', 'bat', 'bat_order', 'b.num', 'inning', 'tm_bat', 'tm_bow', 'city', 'date', 'competition', 'mtype', 'tr', 'extras', 'wkt', 'replacements']
['bow', 'bat', 'bat_order', 'b.num', 'inning', 'tm_bat', 'tm_bow', 'city', 'date', 'competition', 'mtype', 'tr', 'extras', 'wkt', 'replacem

Delhi Capitals
DC vs MI
Mumbai Indians
MI vs DC
['bowler', 'batsman', 'bat_order', 'b.num', 'inning', 'team_batting', 'team_bowling', 'city', 'date', 'competition', 'match_type', 'batsman_runs', 'extras', 'wkt', 'replacements']
(253, 15)
(3744, 15)
['bow', 'bat', 'bat_order', 'b.num', 'inning', 'tm_bat', 'tm_bow', 'city', 'date', 'competition', 'mtype', 'tr', 'extras', 'wkt', 'replacements']
['bow', 'bat', 'bat_order', 'b.num', 'inning', 'tm_bat', 'tm_bow', 'city', 'date', 'competition', 'mtype', 'tr', 'extras', 'wkt', 'replacements']

1359491.yaml
Rajasthan Royals
RR vs CSK
Chennai Super Kings
CSK vs RR
['bowler', 'batsman', 'bat_order', 'b.num', 'inning', 'team_batting', 'team_bowling', 'city', 'date', 'competition', 'match_type', 'batsman_runs', 'extras', 'wkt', 'replacements']
(260, 15)
(3997, 15)
['bow', 'bat', 'bat_order', 'b.num', 'inning', 'tm_bat', 'tm_bow', 'city', 'date', 'competition', 'mtype', 'tr', 'extras', 'wkt', 'replacements']
['bow', 'bat', 'bat_order', 'b.num', 'inn

Punjab Kings
PBKS vs MI
Mumbai Indians
MI vs PBKS
['bowler', 'batsman', 'bat_order', 'b.num', 'inning', 'team_batting', 'team_bowling', 'city', 'date', 'competition', 'match_type', 'batsman_runs', 'extras', 'wkt', 'replacements']
(260, 15)
(7541, 15)
['bow', 'bat', 'bat_order', 'b.num', 'inning', 'tm_bat', 'tm_bow', 'city', 'date', 'competition', 'mtype', 'tr', 'extras', 'wkt', 'replacements']
['bow', 'bat', 'bat_order', 'b.num', 'inning', 'tm_bat', 'tm_bow', 'city', 'date', 'competition', 'mtype', 'tr', 'extras', 'wkt', 'replacements']

1359506.yaml
Royal Challengers Bangalore
RCB vs RR
Rajasthan Royals
RR vs RCB
['bowler', 'batsman', 'bat_order', 'b.num', 'inning', 'team_batting', 'team_bowling', 'city', 'date', 'competition', 'match_type', 'batsman_runs', 'extras', 'wkt', 'replacements']
(256, 15)
(7801, 15)
['bow', 'bat', 'bat_order', 'b.num', 'inning', 'tm_bat', 'tm_bow', 'city', 'date', 'competition', 'mtype', 'tr', 'extras', 'wkt', 'replacements']
['bow', 'bat', 'bat_order', 'b.

Punjab Kings
PBKS vs MI
Mumbai Indians
MI vs PBKS
['bowler', 'batsman', 'bat_order', 'b.num', 'inning', 'team_batting', 'team_bowling', 'city', 'date', 'competition', 'match_type', 'batsman_runs', 'extras', 'wkt', 'replacements']
(257, 15)
(11254, 15)
['bow', 'bat', 'bat_order', 'b.num', 'inning', 'tm_bat', 'tm_bow', 'city', 'date', 'competition', 'mtype', 'tr', 'extras', 'wkt', 'replacements']
['bow', 'bat', 'bat_order', 'b.num', 'inning', 'tm_bat', 'tm_bow', 'city', 'date', 'competition', 'mtype', 'tr', 'extras', 'wkt', 'replacements']

1359521.yaml
Kolkata Knight Riders
KKR vs SRH
Sunrisers Hyderabad
SRH vs KKR
['bowler', 'batsman', 'bat_order', 'b.num', 'inning', 'team_batting', 'team_bowling', 'city', 'date', 'competition', 'match_type', 'batsman_runs', 'extras', 'wkt', 'replacements']
(257, 15)
(11511, 15)
['bow', 'bat', 'bat_order', 'b.num', 'inning', 'tm_bat', 'tm_bow', 'city', 'date', 'competition', 'mtype', 'tr', 'extras', 'wkt', 'replacements']
['bow', 'bat', 'bat_order', 'b

Chennai Super Kings
CSK vs KKR
Kolkata Knight Riders
KKR vs CSK
['bowler', 'batsman', 'bat_order', 'b.num', 'inning', 'team_batting', 'team_bowling', 'city', 'date', 'competition', 'match_type', 'batsman_runs', 'extras', 'wkt', 'replacements']
(253, 15)
(14905, 15)
['bow', 'bat', 'bat_order', 'b.num', 'inning', 'tm_bat', 'tm_bow', 'city', 'date', 'competition', 'mtype', 'tr', 'extras', 'wkt', 'replacements']
['bow', 'bat', 'bat_order', 'b.num', 'inning', 'tm_bat', 'tm_bow', 'city', 'date', 'competition', 'mtype', 'tr', 'extras', 'wkt', 'replacements']

1359536.yaml
Gujarat Titans
GT vs SRH
Sunrisers Hyderabad
SRH vs GT
['bowler', 'batsman', 'bat_order', 'b.num', 'inning', 'team_batting', 'team_bowling', 'city', 'date', 'competition', 'match_type', 'batsman_runs', 'extras', 'wkt', 'replacements']
(255, 15)
(15158, 15)
['bow', 'bat', 'bat_order', 'b.num', 'inning', 'tm_bat', 'tm_bow', 'city', 'date', 'competition', 'mtype', 'tr', 'extras', 'wkt', 'replacements']
['bow', 'bat', 'bat_order

In [6]:

print(all_data.columns.values.tolist())

print(all_data.describe().transpose())

display(all_data.head(3))
#display(all_data[~all_data.wicket.isna()].head(3))


['bow', 'bat', 'bat_order', 'b.num', 'inning', 'tm_bat', 'tm_bow', 'city', 'date', 'competition', 'mtype', 'tr', 'extras', 'wkt', 'replacements']
             count      mean       std  min  25%  50%   75%   max
bat_order  16198.0  3.954315  2.483435  1.0  2.0  3.0   5.0  12.0
b.num      16198.0  9.371584  5.882384  0.0  4.3  9.3  14.4  19.9
tr         16198.0  1.315471  1.712639  0.0  0.0  1.0   1.0   6.0
extras     16198.0  0.067107  0.344840  0.0  0.0  0.0   0.0   5.0
wkt        16198.0  0.049697  0.217326  0.0  0.0  0.0   0.0   1.0


,bow,bat,bat_order,b.num,inning,tm_bat,tm_bow,city,date,competition,mtype,tr,extras,wkt,replacements
0,Mohammed Shami,DP Conway,1,0.1,1st innings,CSK,GT,Ahmedabad,2023-03-31,IPL,T20,0,0,0,NaN
1,Mohammed Shami,DP Conway,1,0.2,1st innings,CSK,GT,Ahmedabad,2023-03-31,IPL,T20,0,1,0,NaN
2,Mohammed Shami,RD Gaikwad,2,0.3,1st innings,CSK,GT,Ahmedabad,2023-03-31,IPL,T20,0,0,0,NaN


In [7]:

print(all_data.columns.values.tolist())

display(all_data.head(11))


['bow', 'bat', 'bat_order', 'b.num', 'inning', 'tm_bat', 'tm_bow', 'city', 'date', 'competition', 'mtype', 'tr', 'extras', 'wkt', 'replacements']


,bow,bat,bat_order,b.num,inning,tm_bat,tm_bow,city,date,competition,mtype,tr,extras,wkt,replacements
0,Mohammed Shami,DP Conway,1,0.1,1st innings,CSK,GT,Ahmedabad,2023-03-31,IPL,T20,0,0,0,NaN
1,Mohammed Shami,DP Conway,1,0.2,1st innings,CSK,GT,Ahmedabad,2023-03-31,IPL,T20,0,1,0,NaN
2,Mohammed Shami,RD Gaikwad,2,0.3,1st innings,CSK,GT,Ahmedabad,2023-03-31,IPL,T20,0,0,0,NaN
3,Mohammed Shami,RD Gaikwad,2,0.4,1st innings,CSK,GT,Ahmedabad,2023-03-31,IPL,T20,1,0,0,NaN
4,Mohammed Shami,DP Conway,1,0.5,1st innings,CSK,GT,Ahmedabad,2023-03-31,IPL,T20,0,0,0,NaN
5,Mohammed Shami,DP Conway,1,0.6,1st innings,CSK,GT,Ahmedabad,2023-03-31,IPL,T20,0,0,0,NaN
6,HH Pandya,RD Gaikwad,2,1.1,1st innings,CSK,GT,Ahmedabad,2023-03-31,IPL,T20,4,0,0,NaN
7,HH Pandya,RD Gaikwad,2,1.2,1st innings,CSK,GT,Ahmedabad,2023-03-31,IPL,T20,0,0,0,NaN
8,HH Pandya,RD Gaikwad,2,1.3,1st innings,CSK,GT,Ahmedabad,2023-03-31,IPL,T20,4,0,0,NaN
9,HH Pandya,RD Gaikwad,2,1.4,1st innings,CSK,GT,Ahmedabad,2023-03-31,IPL,T20,1,0,0,NaN


In [8]:

bat_data_rup = all_data.groupby(['date','city','inning','tm_bat','bat','bat_order'])\
    .agg( {'tr':'sum','wkt':'sum', 'b.num':'count'})\
        .reset_index()\
            .sort_values(by = ['date','city','tm_bat','bat_order','inning'])


In [9]:

print(bat_data_rup.shape)

print(bat_data_rup.columns.values.tolist())


(1552, 9)
['date', 'city', 'inning', 'tm_bat', 'bat', 'bat_order', 'tr', 'wkt', 'b.num']


In [10]:

display(bat_data_rup.tail(50))


,date,city,inning,tm_bat,bat,bat_order,tr,wkt,b.num
1502,2023-05-16,Lucknow,N/A,MI,HR Shokeen,10,0,0,1
1504,2023-05-16,Lucknow,N/A,MI,PP Chawla,11,0,0,1
1503,2023-05-16,Lucknow,N/A,MI,JP Behrendorff,12,0,0,1
1505,2023-05-17,Dharamsala,1st innings,DC,DA Warner,1,46,1,31
1507,2023-05-17,Dharamsala,1st innings,DC,PP Shaw,2,54,1,40
1508,2023-05-17,Dharamsala,1st innings,DC,RR Rossouw,3,82,0,40
1506,2023-05-17,Dharamsala,1st innings,DC,PD Salt,4,26,0,14
1523,2023-05-17,Dharamsala,N/A,DC,Mukesh Kumar,5,0,0,1
1518,2023-05-17,Dharamsala,N/A,DC,AR Patel,6,0,0,1
1519,2023-05-17,Dharamsala,N/A,DC,Aman Hakim Khan,7,0,0,1


In [11]:

print(loc_exp)

fname_exp_alldata= 'QC_batdatarup.xlsx'

bat_data_rup.to_excel(loc_exp + fname_exp_alldata, index = False)


C:\Users\karan\Documents\Cricket-Analytics\IPL\Data\exp\


In [12]:
#### Add more columns to the all_data which might be of use 

In [13]:

# First, we'll need to extract the over from the 'bnum' column.
all_data['over'] = all_data['b.num'].apply(lambda x: int(x)+1)

# Now we can create the 'fpp', 'fmid', 'fdth' columns based on the 'over' column.
all_data['fpp'] = all_data['over'].apply(lambda x: 1 if x <= 6 else 0)
all_data['fmid'] = all_data['over'].apply(lambda x: 1 if 7 <= x <= 16 else 0)
all_data['fdth'] = all_data['over'].apply(lambda x: 1 if x > 16 else 0)

all_data.sort_values(['date','city','tm_bat','bat_order'], inplace = True)


In [14]:

all_data.groupby(['fpp','fmid','fdth']).agg({'over':'nunique'})


over
fpp fmid fdth      
0   0    1        4
    1    0       10
1   0    0        6

In [15]:

print(all_data.columns.values.tolist())

print()
print(all_data.inning.value_counts())

all_data['inn.num'] = all_data['inning'].apply(lambda x : 1 if x =='1st innings' else 2 if x =='2nd innings' else None)

print()
print(all_data['inn.num'].value_counts())


['bow', 'bat', 'bat_order', 'b.num', 'inning', 'tm_bat', 'tm_bow', 'city', 'date', 'competition', 'mtype', 'tr', 'extras', 'wkt', 'replacements', 'over', 'fpp', 'fmid', 'fdth']

1st innings    8112
2nd innings    7564
N/A             522
Name: inning, dtype: int64

1.0    8112
2.0    7564
Name: inn.num, dtype: int64


### Adding the Match Number Column

In [16]:

# Assuming df is your dataframe
all_data['date'] = pd.to_datetime(all_data['date'])  # convert 'date' to datetime format if it's not already

# Create a new column that combines 'city' and 'date'
all_data['city_date'] = all_data['date'].astype(str) + all_data['city'] 

# Sort the DataFrame by 'city_date'
all_data = all_data.sort_values('city_date')

# Use factorize to assign ranks
all_data['m.num'] = pd.factorize(all_data['city_date'])[0] + 1

# Optionally, if you don't need 'city_date' column, you can drop it
all_data = all_data.drop(columns='city_date')


In [17]:

display(all_data.groupby(['m.num']).agg({'b.num':'count'}).reset_index())


,m.num,b.num
0,1,251
1,2,230
2,3,256
3,4,242
4,5,255
...,...,...
60,61,196
61,62,255
62,63,266
63,64,261


### Adding the Match + Over to the data

In [18]:

all_data['m.tbov'] = all_data['m.num']*100 + all_data['inn.num']*10 + all_data['over']


In [19]:

all_data.describe()


,bat_order,b.num,tr,extras,wkt,over,fpp,fmid,fdth,inn.num,m.num,m.tbov
count,16198.000000,16198.000000,16198.000000,16198.000000,16198.000000,16198.000000,16198.000000,16198.000000,16198.000000,15676.000000,16198.000000,15676.000000
mean,3.954315,9.371584,1.315471,0.067107,0.049697,10.019817,0.330720,0.489258,0.180022,1.482521,32.840783,3303.615655
std,2.483435,5.882384,1.712639,0.344840,0.217326,5.860032,0.470487,0.499900,0.384218,0.499710,18.736993,1872.500876
min,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,111.000000
25%,2.000000,4.300000,0.000000,0.000000,0.000000,5.000000,0.000000,0.000000,0.000000,1.000000,17.000000,1717.750000
50%,3.000000,9.300000,1.000000,0.000000,0.000000,10.000000,0.000000,0.000000,0.000000,1.000000,33.000000,3314.000000
75%,5.000000,14.400000,1.000000,0.000000,0.000000,15.000000,1.000000,1.000000,0.000000,2.000000,49.000000,4927.000000
max,12.000000,19.900000,6.000000,5.000000,1.000000,20.000000,1.000000,1.000000,1.000000,2.000000,65.000000,6540.000000


In [20]:

print(all_data.columns.values.tolist())


['bow', 'bat', 'bat_order', 'b.num', 'inning', 'tm_bat', 'tm_bow', 'city', 'date', 'competition', 'mtype', 'tr', 'extras', 'wkt', 'replacements', 'over', 'fpp', 'fmid', 'fdth', 'inn.num', 'm.num', 'm.tbov']


In [21]:

exp_fname = 'bbb_upto_G65_frm_yaml'
exp_ext = '.xlsx'

dropcols = ['extras','replacements']

all_data.drop(columns = dropcols).to_excel(loc_exp + exp_fname + exp_ext , index = False)


### END : Import yaml files